In [7]:
%run homogeneousRelation.ipynb

R = {(1, 4),(1, 5),(2, 4),(3, 6)}
S = {(4, 9),(5, 8),(6, 7)}
R∘S = {(1, 8),(1, 9),(2, 9),(3, 7)}

is binary relation? R = True
is binary relation? S = True

is homogeneous relation? R = False
is homogeneous relation? S = False

is identity relation? R = False
is identity relation? S = False

S[A] = FiniteSet(8, 9)
S-1[A] = FiniteSet(5, 6)

dom S = FiniteSet(4, 5, 6)
rng S = FiniteSet(7, 8, 9)

R == S: False
R != S: True
R < S: False
R <= S: False
R > S: False
R >= S: False

R | S = FiniteSet((1, 4), (1, 5), (2, 4), (3, 6), (4, 9), (5, 8), (6, 7))
R & S = EmptySet
R - S = FiniteSet((1, 4), (1, 5), (2, 4), (3, 6))
R ^ S = FiniteSet((1, 4), (1, 5), (2, 4), (3, 6), (4, 9), (5, 8), (6, 7))
R * S = ProductSet(FiniteSet((1, 4), (1, 5), (2, 4), (3, 6)), FiniteSet((4, 9), (5, 8), (6, 7)))

R complement = {(1, 6),(2, 5),(2, 6),(3, 4),(3, 5)}
R∘S converse = {(7, 3),(8, 1),(9, 1),(9, 2)}

'1' is in relation with: FiniteSet(4, 5) in R
FiniteSet(4, 5)
0
False
False
False
False
True


In [8]:
class PartialOrder(HomogeneousRelation):
    
    def __init__(self,relation=None,domain=None):
        super().__init__(relation, domain)
        
    def __homogeneous_to_partial(self, other):
        return PartialOrder(other.get_relation(), other.get_domain())
    
    def union(self, other):
        return self.__homogeneous_to_partial(super().union(other))
        
    def intersection(self, other):
        return self.__homogeneous_to_partial(super().intersection(other))
        
    def difference(self, other):
        return self.__homogeneous_to_partial(super().difference(other))
    
    def symmetric_difference(self, other):
        return self.__homogeneous_to_partial(super().symmetric_difference(other))
        
    def complement(self):
        return self.__homogeneous_to_partial(super().complement(other))
    
    def composition(self, other):
        return self.__homogeneous_to_partial(super().composition(other))
    
    def converse(self):
        return self.__homogeneous_to_partial(super().converse(other))
        
    def greatest_element(self):
        maximal_elements = self.maximum()
        if len(maximal_elements) != 1:
            return None
        return maximal_elements[0] if self.in_relation_with(maximal_elements[0]) == self.get_domain() else None  
        
    def least_element(self):
        minimal_elements = self.minimum()
        if len(minimal_elements) != 1:
            return None
        return minimal_elements[0] if self.is_in_relation_with(minimal_elements[0]) == self.get_domain() else None  
        
    def maximum(self):
        maximal_elements = []
        for x in self.get_domain():
            if self.is_in_relation_with(x) == {x}:
                maximal_elements.append(x)
        return maximal_elements
        
    def minimum(self):
        minimal_elements = []
        for x in self.get_domain():
            if self.in_relation_with(x) == {x}:
                minimal_elements.append(x)
        return minimal_elements
    
    def is_total_order(self):
        for x in self.get_domain():
            for y in self.get_domain():
                if not((x,y) in self.get_relation() or (y,x) in self.get_relation()):
                    return False
        return True
    
    def infimum(self, args):
        tc = self.transitive_closure()
        result = None
        for x in self.get_domain():
            h = True
            for element in args:
                if x not in tc[element]:
                    h = False
            if h == True:
                if result is None:
                    result = x
                elif x not in tc[result]:
                    result = x
        return result
    
    def supremum(self, args):
        tc = self.transitive_closure()
        result = None
        for x in self.get_domain():
            h = True
            for element in args:
                if element not in tc[x]:
                    h = False
            if h == True:
                if result is None:
                    result = x
                elif result not in tc[x]:
                    result = x
        return result
        
    def is_lattice(self):
        for x,y in self.cartesian_product():
            if self.infimum([x,y]) is None or self.supremum([x,y]) is None:
                return False
        return True
        
    def transitive_closure(self):
        result = dict()
        for x in self.get_domain():
            for y in self.in_relation_with(x):
                if x not in result.keys():
                    result[x]  = FiniteSet(y) | self.in_relation_with(y)
                else:
                    result[x] |= FiniteSet(y) | self.in_relation_with(y) 
        return result
                    
                    
                    
        
    

In [9]:
r = PartialOrder({(1,1),(2,2),(3,3),(2,1),(3,1),(3,2)},{1,2,3})
print(f"R = {r}")

print(r.maximum())
print(r.minimum())
print(r.greatest_element())
print(r.least_element())
print(r.is_partial_order())
print(r.is_total_order())

R = {(1, 1),(2, 1),(2, 2),(3, 1),(3, 2),(3, 3)}
[3]
[1]
3
1
True
True


In [10]:
print(r.in_relation_with(1) == {1})

True


In [11]:
s = FiniteSet(1,2,3,4,5,6,10,12,15,20,30,60)
    
r = s ** 2

br = set()
for a,b in r:
    if b % a == 0:
        br.add((a,b))
        
p = PartialOrder(br, s)


print(p.infimum([4,10]))
print(p.supremum([4,10]))

print(p.infimum([4,6,10]))
print(p.supremum([4,6,10]))

print(p.infimum([4,6,10,15]))
print(p.supremum([4,6,10,15]))

print(p.infimum([60,3]))
print(p.supremum([60,3]))




20
2
60
2
60
1
60
3


In [12]:
#print(f"lattice? -> {p.is_lattice()}")

lattice? -> True
